In [1]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <math.h>

// using a char array as bit field
unsigned char array[125];
unsigned char array2[125];

int main()
{
    // read bit no b
    int b = 2;
    int value = (array[b/8] & (1 << (b % 8)));
    printf("get: %d\n", value);

    // set bit no b
    array[b/8] |= (1 << (b%8));
    value = (array[b/8] & (1 << (b % 8))) != 0;
    printf("set: %d\n", value);

    printf("compare before: %d\n", memcmp(array, array2, sizeof(array)));

    //clear bit no b
    array[b/8] &= ~(1 << (b%8));
    value = (array[b/8] & (1 << (b % 8))) != 0;
    printf("reset: %d\n", value);

    printf("compare after: %d\n", memcmp(array, array2, sizeof(array)));
    
    
    float f = 1.0;
    float result = ldexp(f, 16);
    result += 10;
    printf("float result is: %f\n", result);
    
    
    char aa = 10;
    char bb = 10;
    printf("xor value: %d", aa ^ bb);
    return 0;
}


Writing main.c


In [2]:
!gcc main.c -o main
!./main

get: 0
set: 1
compare before: 4
reset: 0
compare after: 0
float result is: 65546.000000
xor value: 0

In [3]:
%%writefile dominating_query.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <math.h>

#define MAX_LINE_LENGTH     1024
#define MAX_FIELD_LENGTH    128
#define MAX_MOVIE_ID        1001
#define MAX_USER_ID         2001
#define NUM_OF_BUCKETS      11
#define TOPK                3


int ratings[MAX_MOVIE_ID][MAX_USER_ID];
unsigned int bucket[NUM_OF_BUCKETS][3000];
unsigned int result[NUM_OF_BUCKETS * TOPK];
//float result[NUM_OF_BUCKETS * TOPK];
int cur_idx[MAX_MOVIE_ID];
bool user[MAX_USER_ID];


int unique_user;

const double EPSILON = 1e-9;
int compareWeightedScore(const void *a, const void *b) {
    const float *record1 = (const float *) a;
    const float *record2 = (const float *) b;

    const float score1 = ldexp(*record1, -16);
    const float score2 = ldexp(*record2, -16);

    if(fabs(score1 - score2) < EPSILON) {
        return 0;
    } else if(score1 > score2) {
        return -1;
    } else {
        return 1;
    }
}

int compareScore(const void *a, const void *b) {
    const unsigned int *num1 = (const unsigned int *)a;
    const unsigned int *num2 = (const unsigned int *)b;
    if (*num1 > *num2) {
        return -1; // num1 comes before num2
    } else if (*num1 < *num2) {
        return 1;  // num2 comes before num1
    } else {
        return 0;  // Both numbers are equal
    }
}

int main()
{
    /* final.csv is sorted by movieId and then by userId. For each group of movieId the first 20 rows are picked. */
    FILE* file = fopen("/kaggle/input/elementarydataset/final.csv", "r");
    //FILE* file = fopen("final.csv", "r");
    if (file == NULL) {
        printf("Failed to open the file.\n");
        return 0;
    }
    memset(ratings, -1, sizeof(ratings));
//    for(int i = 0; i < MAX_MOVIE_ID; ++i) {
//        for(int j = 0; j < MAX_USER_ID; ++j) {
//            ratings[i][j] = -1.00;
//        }
//    }
    char line[MAX_LINE_LENGTH];
    int id_no = -1;
    while (fgets(line, sizeof(line), file)) {
        id_no++;
        if(id_no == 0) continue; // the first row has the names of the columns. omit that.
        int userId, movieId;
        int rating;
        sscanf(line, "%d,%d,%d", &userId, &movieId, &rating);
        if(movieId >= MAX_MOVIE_ID || userId >= MAX_USER_ID) continue;
        if(user[userId] == false) {
            user[userId] = true;
            unique_user += 1;
        }
//        printf("%d %d %d\n", movieId, userId, rating);
        ratings[movieId][userId] = rating;
    }
    fclose(file);

//    for(int i = 0; i < 10; ++i) {
//        for(int j = 0; j < 10; ++j) {
//            printf("MovieId: %d, UserId: %d, Rating: %d\n", i, j, ratings[i][j]);
//        }
//    }


    memset(cur_idx, 0, sizeof(cur_idx));

    for(unsigned int movie_id = 1; movie_id < MAX_MOVIE_ID; ++movie_id) {
        unsigned int sum = 0;
        for(int user_id = 1; user_id < MAX_USER_ID; ++user_id) {
            if(ratings[movie_id][user_id] == -1) continue;
            sum += ratings[movie_id][user_id];
        }
        printf("sum : %d\n", sum);
        unsigned int score = movie_id;
        score |= (sum << 16);
//        char buffer[33];
//        itoa(score, buffer, 2);
//        printf("binary: %s\n", buffer);
        unsigned int bucket_id = movie_id % NUM_OF_BUCKETS;
        bucket[bucket_id][cur_idx[bucket_id]] = score;
        cur_idx[bucket_id]++;
        printf("movie_id: %d cur_idx: %d\n", movie_id, cur_idx[bucket_id]);
    }
    for(int i = 0; i < 10; ++i) {
        printf("%d\n", (bucket[1][i] << 16) >> 16);
    }
    for(int i = 0; i < NUM_OF_BUCKETS; ++i) {
        printf("%d, ", cur_idx[i]);
        qsort(bucket[i], (size_t) cur_idx[i], sizeof(unsigned int), compareScore);
    }
    printf("\n");
//    for(int i = 0; i < cur_idx[0]; ++i) {
//        printf("%d\n", bucket[0][i]);
//    }
    int at = 0;
    for(int i = 0; i < NUM_OF_BUCKETS; ++i) {
        for(int top = 0; top < TOPK; ++top) {
            result[at] = bucket[i][top];
            at++;
        }
    }

    int TKD[at];
    memset(TKD, 0, sizeof(TKD));
    // calculate TDK scores for NUM_OF_BUCKETS * TOPK movies
    for(int i = 0; i < at; ++i) {
        int p = ((result[i] << 16) >> 16);
        for(int j = 0; j < at; ++j) {
            if(i == j) continue; // dont need to compare with itself
            int q = ((result[j] << 16) >> 16);
            bool all_greater_or_equal = true;
            bool at_least_one_greater = false;
            for(int k = 1; k < MAX_USER_ID; ++k) {
                all_greater_or_equal &= (ratings[p][k] >= ratings[q][k]);
                at_least_one_greater |= (ratings[p][k] > ratings[q][k]);
            }
            if(all_greater_or_equal && at_least_one_greater) {
                TKD[i] += 1;
            }
        }
    }

    float weight_factor[at];
    // calculate weights for NUM_OF_BUCKETS * TOPK movies
    printf("no of unique users: %d\n", unique_user);
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
        int review_count = 0;
        for(int user_id = 0; user_id < MAX_USER_ID; ++user_id) {
            review_count += (ratings[movie_id][user_id] != -1);
        }
        printf("Review count for movie_id: %d is: %d\n", movie_id, review_count);
        weight_factor[i] = (float) review_count / (float) unique_user;
    }

    float weighted_tkd_score[at];
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
//        printf("id: %d, tkd_score: %d, weight_factor: %f\n", movie_id, TKD[i], weight_factor[i]);
        float weighted_score = (float) TKD[i] * weight_factor[i];
        weighted_tkd_score[i] = ldexp(weighted_score, 16);
//        weighted_tkd_score[i] += idexp(movie_id, 0);
        weighted_tkd_score[i] += (float) movie_id;
        printf("weighted_tkd_score: %f\n", weighted_tkd_score[i]);
    }

    // Problem below

    qsort(weighted_tkd_score, at, sizeof(float), compareWeightedScore);
    for(int i = 0; i < TOPK; ++i) {
        printf("weighted tkd score: %f\n", ldexp(weighted_tkd_score[i], -16));
    }


//    qsort(result, NUM_OF_BUCKETS * TOPK, sizeof(unsigned int), compareScore);
//    printf("Top-K results are: \n");
//    for(int i = 0; i < TOPK; ++i) {
//        printf("score: %d, id: %d\n", (result[i] >> 16), ((result[i] << 16) >> 16));
//    }
    return 0;
}



Writing dominating_query.c


In [4]:
!gcc dominating_query.c -o dominating_query
!./dominating_query

sum : 79
movie_id: 1 cur_idx: 1
sum : 54
movie_id: 2 cur_idx: 1
sum : 65
movie_id: 3 cur_idx: 1
sum : 61
movie_id: 4 cur_idx: 1
sum : 59
movie_id: 5 cur_idx: 1
sum : 75
movie_id: 6 cur_idx: 1
sum : 72
movie_id: 7 cur_idx: 1
sum : 61
movie_id: 8 cur_idx: 1
sum : 59
movie_id: 9 cur_idx: 1
sum : 66
movie_id: 10 cur_idx: 1
sum : 73
movie_id: 11 cur_idx: 1
sum : 52
movie_id: 12 cur_idx: 2
sum : 60
movie_id: 13 cur_idx: 2
sum : 70
movie_id: 14 cur_idx: 2
sum : 47
movie_id: 15 cur_idx: 2
sum : 75
movie_id: 16 cur_idx: 2
sum : 73
movie_id: 17 cur_idx: 2
sum : 61
movie_id: 18 cur_idx: 2
sum : 52
movie_id: 19 cur_idx: 2
sum : 58
movie_id: 20 cur_idx: 2
sum : 70
movie_id: 21 cur_idx: 2
sum : 66
movie_id: 22 cur_idx: 2
sum : 57
movie_id: 23 cur_idx: 3
sum : 63
movie_id: 24 cur_idx: 3
sum : 82
movie_id: 25 cur_idx: 3
sum : 73
movie_id: 26 cur_idx: 3
sum : 57
movie_id: 27 cur_idx: 3
sum : 79
movie_id: 28 cur_idx: 3
sum : 81
movie_id: 29 cur_idx: 3
sum : 53
movie_id: 30 cur_idx: 3
sum : 66
movie_id: 

In [5]:
%%writefile bit-bucketing.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <math.h>

#define MAX_LINE_LENGTH     1024
#define MAX_FIELD_LENGTH    128
#define MAX_MOVIE_ID        1001
#define MAX_USER_ID         2001
#define NUM_OF_BUCKETS      11
#define TOPK                3


int ratings[MAX_MOVIE_ID][MAX_USER_ID];
unsigned int bucket[NUM_OF_BUCKETS][3000];
unsigned int result[NUM_OF_BUCKETS * TOPK];
char bitbucket[MAX_MOVIE_ID][126]; // 1000 contiguous bits for generating bucket id

int cur_idx[MAX_MOVIE_ID];
int movie_key_list[MAX_MOVIE_ID];

bool user[MAX_USER_ID];
bool movie[MAX_MOVIE_ID];

int unique_user;
int unique_movie;
int index_movie_key_list;


struct WeightedTKD {
    float weighted_score;
    int movieId;
};

int compareScore(const void *a, const void *b) {
    const unsigned int *num1 = (const unsigned int *)a;
    const unsigned int *num2 = (const unsigned int *)b;
    if (*num1 > *num2) {
        return -1; // num1 comes before num2
    } else if (*num1 < *num2) {
        return 1;  // num2 comes before num1
    } else {
        return 0;  // Both numbers are equal
    }
}

const double EPSILON = 1e-9;
int compareFloats(const void *a, const void *b) {
    const struct WeightedTKD *floatA = (const struct WeightedTKD *)a;
    const struct WeightedTKD *floatB = (const struct WeightedTKD *)b;

    // Compare using tolerance to handle rounding errors
    if (fabs(floatA->weighted_score - floatB->weighted_score) < EPSILON) {
        return 0;  // Considered approximately equal
    } else if (floatA->weighted_score > floatB->weighted_score) {
        return -1;
    } else {
        return 1;
    }
}


int main()
{
    /* final.csv is sorted by movieId and then by userId. For each group of movieId the first 20 rows are picked. */
//    FILE* file = fopen("final.csv", "r");
    FILE* file = fopen("/kaggle/input/testset/test.csv", "r");
    if (file == NULL) {
        printf("Failed to open the file.\n");
        return 0;
    }
    memset(ratings, -1, sizeof(ratings));

    char line[MAX_LINE_LENGTH];
    int id_no = -1;
    while (fgets(line, sizeof(line), file)) {
        id_no++;
        if(id_no == 0) continue; // the first row has the names of the columns. omit that.
        int userId, movieId;
        int rating;
        sscanf(line, "%d,%d,%d", &userId, &movieId, &rating);
        if(movieId >= MAX_MOVIE_ID || userId >= MAX_USER_ID) continue;
        if(user[userId] == false) {
            user[userId] = true;
            unique_user += 1;
        }

        bitbucket[movieId][userId / 8] |= (1 << (userId % 8));
        if(movie[movieId] == false) {
            movie[movieId] = true;
            unique_movie += 1;
            movie_key_list[index_movie_key_list] = movieId;
            index_movie_key_list += 1;
        }

        printf("%d %d %d\n", movieId, userId, rating);
        ratings[movieId][userId] = rating;
    }
    fclose(file);

//    printf("listed movies are: ");
//    for(int i = 0; i < index_movie_key_list; ++i) {
//        printf("%d ", movie_key_list[i]);
//    }
//    printf("\n");
//    printf("decimal value for the key is: %d", bitbucket[3][0]);
//
//    return 0;

    bool bucket_seen[index_movie_key_list];
    memset(bucket_seen, 0, sizeof(bucket_seen));

    int BUCKET[index_movie_key_list][index_movie_key_list];

    int cur_idx_for_this_bucket[index_movie_key_list];
    memset(cur_idx_for_this_bucket, 0, sizeof(cur_idx_for_this_bucket));


    for(int i = 0; i < index_movie_key_list; ++i) {
        if(bucket_seen[i]) continue;
        for(int j = 0; j < index_movie_key_list; ++j) {
            if(bucket_seen[j]) continue;

            if(!memcmp(bitbucket[movie_key_list[i]], bitbucket[movie_key_list[j]], sizeof(bitbucket[0]))) {
                BUCKET[i][cur_idx_for_this_bucket[i]] = movie_key_list[j];
                cur_idx_for_this_bucket[i] += 1;
                bucket_seen[j] = true;
            }
        }
    }

    memset(cur_idx, 0, sizeof(cur_idx));
    int bucket_id = 0;

    for(int i = 0; i < index_movie_key_list; ++i) {
        if(bucket_seen[i] == false) continue;
        printf("This bucket has the movieIds: \n");
        for(int j = 0; j < cur_idx_for_this_bucket[i]; ++j) {
            printf("%d ", BUCKET[i][j]);
            unsigned int sum = 0;
            int movie_id = BUCKET[i][j];
            for(int user_id = 1; user_id < MAX_USER_ID; ++user_id) {
                if(ratings[movie_id][user_id] == -1) continue;
                sum += ratings[movie_id][user_id];
            }
            unsigned int score = movie_id; // BUCKET[i][j] is the movieId
            score |= (sum << 16);

            bucket[bucket_id][cur_idx[bucket_id]] = score;
            cur_idx[bucket_id] += 1;
        }
        bucket_id += 1;
        printf("\n");
    }

//    for(int i = 0; i < 5; ++i) {
//        printf("%d\n", cur_idx[i]);
//    }

//    for(unsigned int movie_id = 1; movie_id < MAX_MOVIE_ID; ++movie_id) {
//        unsigned int sum = 0;
//        for(int user_id = 1; user_id < MAX_USER_ID; ++user_id) {
//            if(ratings[movie_id][user_id] == -1) continue;
//            sum += ratings[movie_id][user_id];
//        }
//        printf("sum : %d\n", sum);
//        unsigned int score = movie_id;
//        score |= (sum << 16);
//
//        unsigned int bucket_id = movie_id % NUM_OF_BUCKETS;
//        bucket[bucket_id][cur_idx[bucket_id]] = score;
//        cur_idx[bucket_id]++;
//        printf("movie_id: %d cur_idx: %d\n", movie_id, cur_idx[bucket_id]);
//    }
//    for(int i = 0; i < 10; ++i) {
//        printf("%d\n", (bucket[1][i] << 16) >> 16);
//    }

    int BUCKETS_REQ = bucket_id;
    for(int i = 0; i < BUCKETS_REQ; ++i) {
        printf("%d, ", cur_idx[i]);
        qsort(bucket[i], (size_t) cur_idx[i], sizeof(unsigned int), compareScore);
    }
    printf("\n");
//    for(int i = 0; i < cur_idx[0]; ++i) {
//        printf("%d\n", bucket[0][i]);
//    }
    int at = 0;
    for(int i = 0; i < BUCKETS_REQ; ++i) {
        for(int top = 0; top < TOPK && top < cur_idx[i]; ++top) {
            result[at] = bucket[i][top];
            at++;
        }
    }

    int TKD[at];
    memset(TKD, 0, sizeof(TKD));
    // calculate TDK scores for NUM_OF_BUCKETS * TOPK movies
    for(int i = 0; i < at; ++i) {
        int p = ((result[i] << 16) >> 16);
        for(int j = 0; j < at; ++j) {
            if(i == j) continue; // dont need to compare with itself
            int q = ((result[j] << 16) >> 16);
            bool all_greater_or_equal = true;
            bool at_least_one_greater = false;
            for(int k = 1; k < MAX_USER_ID; ++k) {
                all_greater_or_equal &= (ratings[p][k] >= ratings[q][k]);
                at_least_one_greater |= (ratings[p][k] > ratings[q][k]);
            }
            if(all_greater_or_equal && at_least_one_greater) {
                printf("EXECUTED\n\n\n");
                TKD[i] += 1;
            }
        }
    }

    float weight_factor[at];
    // calculate weights for NUM_OF_BUCKETS * TOPK movies
    printf("no of unique users: %d\n", unique_user);
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
        int review_count = 0;
        for(int user_id = 0; user_id < MAX_USER_ID; ++user_id) {
            review_count += (ratings[movie_id][user_id] != -1);
        }
        printf("Review count for movie_id: %d is: %d\n", movie_id, review_count);
        weight_factor[i] = (float) review_count / (float) unique_user;
    }

    qsort(result, BUCKETS_REQ * TOPK, sizeof(unsigned int), compareScore);
    printf("Top-K results are: \n");
    for(int i = 0; i < TOPK; ++i) {
        printf("score: %d, id: %d\n", (result[i] >> 16), ((result[i] << 16) >> 16));
    }

    struct WeightedTKD final_score[at];
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
        printf("id: %d, tkd_score: %d, weight_factor: %f\n", movie_id, TKD[i], weight_factor[i]);
        float weighted_score = (float) TKD[i] * weight_factor[i];

        printf("This weighted_score: %f\n", weighted_score);
        final_score[i].weighted_score = weighted_score;
        final_score[i].movieId = movie_id;
        printf("weighted_tkd_score: %f\n", final_score[i].weighted_score);

        printf("\n\n");
    }

//    qsort(WEIGHTED_TKD + 1, MAX_MOVIE_ID, sizeof(struct WeightedTKD), compareFloats);

    qsort(final_score, at, sizeof(struct WeightedTKD), compareFloats);

    printf("Top-K results are: \n");
    for(int i = 0; i < TOPK; ++i) {
        printf("id: %d, score: %f\n", final_score[i].movieId, final_score[i].weighted_score);
    }
    return 0;
}


Writing bit-bucketing.c


In [6]:
!gcc bit-bucketing.c -o main
!./main

1 1 4
1 2 3
2 1 3
2 2 4
2 3 4
3 3 5
4 1 3
4 2 4
This bucket has the movieIds: 
1 4 
This bucket has the movieIds: 
2 
This bucket has the movieIds: 
3 
This bucket has the movieIds: 

2, 1, 1, 0, 
EXECUTED


no of unique users: 3
Review count for movie_id: 4 is: 2
Review count for movie_id: 1 is: 2
Review count for movie_id: 2 is: 3
Review count for movie_id: 3 is: 1
Top-K results are: 
score: 11, id: 2
score: 7, id: 4
score: 7, id: 1
id: 2, tkd_score: 0, weight_factor: 0.666667
This weighted_score: 0.000000
weighted_tkd_score: 0.000000


id: 4, tkd_score: 0, weight_factor: 0.666667
This weighted_score: 0.000000
weighted_tkd_score: 0.000000


id: 1, tkd_score: 1, weight_factor: 1.000000
This weighted_score: 1.000000
weighted_tkd_score: 1.000000


id: 3, tkd_score: 0, weight_factor: 0.333333
This weighted_score: 0.000000
weighted_tkd_score: 0.000000


Top-K results are: 
id: 1, score: 1.000000
id: 2, score: 0.000000
id: 4, score: 0.000000


In [7]:
%%writefile parallel-bit-bucketing.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <math.h>

#define MAX_LINE_LENGTH     1024
#define MAX_FIELD_LENGTH    128
#define MAX_MOVIE_ID        1001
#define MAX_USER_ID         2001
#define NUM_OF_BUCKETS      11
#define TOPK                3


int ratings[MAX_MOVIE_ID][MAX_USER_ID];
unsigned int bucket[NUM_OF_BUCKETS][3000];
unsigned int result[NUM_OF_BUCKETS * TOPK];
char bitbucket[MAX_MOVIE_ID][126]; // 1000 contiguous bits for generating bucket id

int cur_idx[MAX_MOVIE_ID];
int movie_key_list[MAX_MOVIE_ID];

bool user[MAX_USER_ID];
bool movie[MAX_MOVIE_ID];

int unique_user;
int unique_movie;
int index_movie_key_list;


struct WeightedTKD {
    float weighted_score;
    int movieId;
};

int compareScore(const void *a, const void *b) {
    const unsigned int *num1 = (const unsigned int *)a;
    const unsigned int *num2 = (const unsigned int *)b;
    if (*num1 > *num2) {
        return -1; // num1 comes before num2
    } else if (*num1 < *num2) {
        return 1;  // num2 comes before num1
    } else {
        return 0;  // Both numbers are equal
    }
}

const double EPSILON = 1e-9;
int compareFloats(const void *a, const void *b) {
    const struct WeightedTKD *floatA = (const struct WeightedTKD *)a;
    const struct WeightedTKD *floatB = (const struct WeightedTKD *)b;

    // Compare using tolerance to handle rounding errors
    if (fabs(floatA->weighted_score - floatB->weighted_score) < EPSILON) {
        return 0;  // Considered approximately equal
    } else if (floatA->weighted_score > floatB->weighted_score) {
        return -1;
    } else {
        return 1;
    }
}

__global__ void gpu_bit_bucketing(int *BUCKET, char *bitbucket, int *movie_key_list, int *index_movie_key_list, int *cur_idx_for_this_bucket, bool *bucket_seen) {
    
    int minIndex = threadIdx.x + blockIdx.x * blockDim.x;
    int maxIndex = minIndex + blockDim.x;
    int maxAllowed = *index_movie_key_list;
    
    for(int i = minIndex; i < maxIndex && i < maxAllowed; ++i) {
        if(bucket_seen[i]) continue;
        for(int j = minIndex; j < maxIndex && j < maxAllowed; ++j) {
            if(bucket_seen[j]) continue;
            
            if(!(bitbucket[movie_key_list[i]] ^ bitbucket[movie_key_list[j]])) {
               
                atomicExch(&BUCKET[i][cur_idx_for_this_bucket[i], movie_key_list[j]]);
                atomicAdd(&cur_idx_for_this_bucket[i], 1);
               
                //BUCKET[i][cur_idx_for_this_bucket[i]] = movie_key_list[j];
                //cur_idx_for_this_bucket[i] += 1;
                
                bucket_seen[j] = true;
            }
        }
    }
}

int main()
{
    /* final.csv is sorted by movieId and then by userId. For each group of movieId the first 20 rows are picked. */
//    FILE* file = fopen("final.csv", "r");
    FILE* file = fopen("/kaggle/input/testset/test.csv", "r");
    if (file == NULL) {
        printf("Failed to open the file.\n");
        return 0;
    }
    memset(ratings, -1, sizeof(ratings));

    char line[MAX_LINE_LENGTH];
    int id_no = -1;
    while (fgets(line, sizeof(line), file)) {
        id_no++;
        if(id_no == 0) continue; // the first row has the names of the columns. omit that.
        int userId, movieId;
        int rating;
        sscanf(line, "%d,%d,%d", &userId, &movieId, &rating);
        if(movieId >= MAX_MOVIE_ID || userId >= MAX_USER_ID) continue;
        if(user[userId] == false) {
            user[userId] = true;
            unique_user += 1;
        }

        bitbucket[movieId][userId / 8] |= (1 << (userId % 8));
        if(movie[movieId] == false) {
            movie[movieId] = true;
            unique_movie += 1;
            movie_key_list[index_movie_key_list] = movieId;
            index_movie_key_list += 1;
        }

        printf("%d %d %d\n", movieId, userId, rating);
        ratings[movieId][userId] = rating;
    }
    fclose(file);

//    printf("listed movies are: ");
//    for(int i = 0; i < index_movie_key_list; ++i) {
//        printf("%d ", movie_key_list[i]);
//    }
//    printf("\n");
//    printf("decimal value for the key is: %d", bitbucket[3][0]);
//
//    return 0;

    bool bucket_seen[index_movie_key_list];
    memset(bucket_seen, 0, sizeof(bucket_seen));

    int BUCKET[index_movie_key_list][index_movie_key_list];

    int cur_idx_for_this_bucket[index_movie_key_list];
    memset(cur_idx_for_this_bucket, 0, sizeof(cur_idx_for_this_bucket));
    
    
    // parallel bit-bucketing-start
    
    int *dBUCKET, *dmovie_key_list, *dindex_movie_key_list, *dcur_idx_for_this_bucket;
    char *dbitbucket;
    bool *dbucket_seen;
    
    cudaMalloc((void **) &dBUCKET, index_movie_key_list * index_movie_key_list * sizeof(int));
    cudaMalloc((void **) &dbitbucket, MAX_MOVIE_ID * 126 * sizeof(char));
    cudaMalloc((void **) &dmovie_key_list, MAX_MOVIE_ID * sizeof(int));
    cudaMalloc((void **) &dcur_idx_for_this_bucket, MAX_MOVIE_ID * sizeof(int));
    cudaMalloc((void **) &dbucket_seen, index_movie_key_list * sizeof(bool));
    cudaMalloc((void **) &dindex_movie_key_list, sizeof(int));
    
    
    cudaMemcpy(dBUCKET, BUCKET, index_movie_key_list * index_movie_key_list * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dbitbucket, bitbucket, MAX_MOVIE_ID * 126 * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(dmovie_key_list, movie_key_list, MAX_MOVIE_ID * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dcur_idx_for_this_bucket, cur_idx_for_this_bucket, MAX_MOVIE_ID * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dbucket_seen, bucket_seen, index_movie_key_list * sizeof(bool), cudaMemcpyHostToDevice);
    cudaMemcpy(dindex_movie_key_list, &index_movie_key_list, sizeof(int), cudaMemcpyHostToDevice);
    
    // kernel call
    gpu_bit_bucketing<<<2, 1>>>(dBUCKET, dbitbucket, dmovie_key_list, dindex_movie_key_list, dcur_idx_for_this_bucket, dbucket_seen);
    
    cudaMemcpy(BUCKET, dBUCKET, index_movie_key_list * index_movie_key_list * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(cur_idx_for_this_bucket, dcur_idx_for_this_bucket, MAX_MOVIE_ID * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(bucket_seen, dbucket_seen, index_movie_key_list * sizeof(bool), cudaMemcpyDeviceToHost);
    cudaDeviceSynchronize();
    
    // parallel-bit-bucketing-end
    
    // serial-bit-bucketing-start
    
    for(int i = 0; i < index_movie_key_list; ++i) {
        if(bucket_seen[i]) continue;
        for(int j = 0; j < index_movie_key_list; ++j) {
            if(bucket_seen[j]) continue;

            if(!memcmp(bitbucket[movie_key_list[i]], bitbucket[movie_key_list[j]], sizeof(bitbucket[0]))) {
                BUCKET[i][cur_idx_for_this_bucket[i]] = movie_key_list[j];
                cur_idx_for_this_bucket[i] += 1;
                bucket_seen[j] = true;
            }
        }
    }

    // serial-bit-bucketing-end
    
    memset(cur_idx, 0, sizeof(cur_idx));
    int bucket_id = 0;

    for(int i = 0; i < index_movie_key_list; ++i) {
        if(bucket_seen[i] == false) continue;
        printf("This bucket has the movieIds: \n");
        for(int j = 0; j < cur_idx_for_this_bucket[i]; ++j) {
            printf("%d ", BUCKET[i][j]);
            unsigned int sum = 0;
            int movie_id = BUCKET[i][j];
            for(int user_id = 1; user_id < MAX_USER_ID; ++user_id) {
                if(ratings[movie_id][user_id] == -1) continue;
                sum += ratings[movie_id][user_id];
            }
            unsigned int score = movie_id; // BUCKET[i][j] is the movieId
            score |= (sum << 16);

            bucket[bucket_id][cur_idx[bucket_id]] = score;
            cur_idx[bucket_id] += 1;
        }
        bucket_id += 1;
        printf("\n");
    }

//    for(int i = 0; i < 5; ++i) {
//        printf("%d\n", cur_idx[i]);
//    }

//    for(unsigned int movie_id = 1; movie_id < MAX_MOVIE_ID; ++movie_id) {
//        unsigned int sum = 0;
//        for(int user_id = 1; user_id < MAX_USER_ID; ++user_id) {
//            if(ratings[movie_id][user_id] == -1) continue;
//            sum += ratings[movie_id][user_id];
//        }
//        printf("sum : %d\n", sum);
//        unsigned int score = movie_id;
//        score |= (sum << 16);
//
//        unsigned int bucket_id = movie_id % NUM_OF_BUCKETS;
//        bucket[bucket_id][cur_idx[bucket_id]] = score;
//        cur_idx[bucket_id]++;
//        printf("movie_id: %d cur_idx: %d\n", movie_id, cur_idx[bucket_id]);
//    }
//    for(int i = 0; i < 10; ++i) {
//        printf("%d\n", (bucket[1][i] << 16) >> 16);
//    }

    int BUCKETS_REQ = bucket_id;
    for(int i = 0; i < BUCKETS_REQ; ++i) {
        printf("%d, ", cur_idx[i]);
        qsort(bucket[i], (size_t) cur_idx[i], sizeof(unsigned int), compareScore);
    }
    printf("\n");
//    for(int i = 0; i < cur_idx[0]; ++i) {
//        printf("%d\n", bucket[0][i]);
//    }
    int at = 0;
    for(int i = 0; i < BUCKETS_REQ; ++i) {
        for(int top = 0; top < TOPK && top < cur_idx[i]; ++top) {
            result[at] = bucket[i][top];
            at++;
        }
    }

    int TKD[at];
    memset(TKD, 0, sizeof(TKD));
    // calculate TDK scores for NUM_OF_BUCKETS * TOPK movies
    for(int i = 0; i < at; ++i) {
        int p = ((result[i] << 16) >> 16);
        for(int j = 0; j < at; ++j) {
            if(i == j) continue; // dont need to compare with itself
            int q = ((result[j] << 16) >> 16);
            bool all_greater_or_equal = true;
            bool at_least_one_greater = false;
            for(int k = 1; k < MAX_USER_ID; ++k) {
                all_greater_or_equal &= (ratings[p][k] >= ratings[q][k]);
                at_least_one_greater |= (ratings[p][k] > ratings[q][k]);
            }
            if(all_greater_or_equal && at_least_one_greater) {
                printf("EXECUTED\n\n\n");
                TKD[i] += 1;
            }
        }
    }

    float weight_factor[at];
    // calculate weights for NUM_OF_BUCKETS * TOPK movies
    printf("no of unique users: %d\n", unique_user);
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
        int review_count = 0;
        for(int user_id = 0; user_id < MAX_USER_ID; ++user_id) {
            review_count += (ratings[movie_id][user_id] != -1);
        }
        printf("Review count for movie_id: %d is: %d\n", movie_id, review_count);
        weight_factor[i] = (float) review_count / (float) unique_user;
    }

    qsort(result, BUCKETS_REQ * TOPK, sizeof(unsigned int), compareScore);
    printf("Top-K results are: \n");
    for(int i = 0; i < TOPK; ++i) {
        printf("score: %d, id: %d\n", (result[i] >> 16), ((result[i] << 16) >> 16));
    }

    struct WeightedTKD final_score[at];
    for(int i = 0; i < at; ++i) {
        int movie_id = ((result[i] << 16) >> 16);
        printf("id: %d, tkd_score: %d, weight_factor: %f\n", movie_id, TKD[i], weight_factor[i]);
        float weighted_score = (float) TKD[i] * weight_factor[i];

        printf("This weighted_score: %f\n", weighted_score);
        final_score[i].weighted_score = weighted_score;
        final_score[i].movieId = movie_id;
        printf("weighted_tkd_score: %f\n", final_score[i].weighted_score);

        printf("\n\n");
    }

//    qsort(WEIGHTED_TKD + 1, MAX_MOVIE_ID, sizeof(struct WeightedTKD), compareFloats);

    qsort(final_score, at, sizeof(struct WeightedTKD), compareFloats);

    printf("Top-K results are: \n");
    for(int i = 0; i < TOPK; ++i) {
        printf("id: %d, score: %f\n", final_score[i].movieId, final_score[i].weighted_score);
    }
    return 0;
}


Writing parallel-bit-bucketing.cu


In [8]:
!nvcc -o main parallel-bit-bucketing.cu
!./main

parallel-bit-bucketing.cu(76): warning #174-D: expression has no effect

parallel-bit-bucketing.cu(76): error: expression must have pointer-to-object type but it has type "int"

parallel-bit-bucketing.cu(76): error: no instance of overloaded function "atomicExch" matches the argument list
            argument types are: (<error-type>)

2 errors detected in the compilation of "parallel-bit-bucketing.cu".
1 1 4
1 2 3
2 1 3
2 2 4
2 3 4
3 3 5
4 1 3
4 2 4
This bucket has the movieIds: 
1 4 
This bucket has the movieIds: 
2 
This bucket has the movieIds: 
3 
This bucket has the movieIds: 

2, 1, 1, 0, 
EXECUTED


no of unique users: 3
Review count for movie_id: 4 is: 2
Review count for movie_id: 1 is: 2
Review count for movie_id: 2 is: 3
Review count for movie_id: 3 is: 1
Top-K results are: 
score: 11, id: 2
score: 7, id: 4
score: 7, id: 1
id: 2, tkd_score: 0, weight_factor: 0.666667
This weighted_score: 0.000000
weighted_tkd_score: 0.000000


id: 4, tkd_score: 0, weight_factor: 0.666667
Thi

In [9]:
%%writefile loops.cu

#include <stdio.h>
#include <stdlib.h>

__global__ void gpu_cmp(int *a) {
    int index = threadIdx.x + blockIdx.x * blockDim.x;
    int initial = index;
    if(index < 9) {
        for(int i = initial; i < initial + blockDim.x; ++i) {
            for(int j = initial; j < initial + blockDim.x; ++j) {
                if(!memcmp(a[i], a[j], sizeof(int))) {

                }
            }
        }
    }
}

int main() {
    int a[3][3] = {{1, 2, 3}, {1, 2, 3}, {1, 2, 3}};
    
    int *da;
    cudaMalloc((void**) &da, 3 * 3 * sizeof(int));
    
    cudaMemcpy(da, a, 3 * 3 * sizeof(int), cudaMemcpyHostToDevice);
    
    gpu_cmp<<<3, 3>>>(da);
    return 0;
}

Writing loops.cu


In [10]:
!nvcc -o loops loops.cu
!./loops

loops.cu(11): error: argument of type "int" is incompatible with parameter of type "const void *"

loops.cu(11): error: argument of type "int" is incompatible with parameter of type "const void *"

2 errors detected in the compilation of "loops.cu".
/bin/bash: line 1: ./loops: No such file or directory


In [11]:
%%writefile parallel-bit-bucketing.cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdbool.h>
#include <math.h>

#define MAX_LINE_LENGTH     1024
#define MAX_FIELD_LENGTH    128
#define MAX_MOVIE_ID        1001
#define MAX_USER_ID         2001
#define NUM_OF_BUCKETS      11
#define TOPK                3


int ratings[MAX_MOVIE_ID][MAX_USER_ID];
unsigned int bucket[NUM_OF_BUCKETS][3000];
unsigned int result[NUM_OF_BUCKETS * TOPK];
char bitbucket[MAX_MOVIE_ID][126]; // 1000 contiguous bits for generating bucket id

int cur_idx[MAX_MOVIE_ID];
int movie_key_list[MAX_MOVIE_ID];

bool user[MAX_USER_ID];
bool movie[MAX_MOVIE_ID];

int unique_user;
int unique_movie;
int index_movie_key_list;


struct WeightedTKD {
    float weighted_score;
    int movieId;
};

int compareScore(const void *a, const void *b) {
    const unsigned int *num1 = (const unsigned int *)a;
    const unsigned int *num2 = (const unsigned int *)b;
    if (*num1 > *num2) {
        return -1; // num1 comes before num2
    } else if (*num1 < *num2) {
        return 1;  // num2 comes before num1
    } else {
        return 0;  // Both numbers are equal
    }
}

const double EPSILON = 1e-9;
int compareFloats(const void *a, const void *b) {
    const struct WeightedTKD *floatA = (const struct WeightedTKD *)a;
    const struct WeightedTKD *floatB = (const struct WeightedTKD *)b;

    // Compare using tolerance to handle rounding errors
    if (fabs(floatA->weighted_score - floatB->weighted_score) < EPSILON) {
        return 0;  // Considered approximately equal
    } else if (floatA->weighted_score > floatB->weighted_score) {
        return -1;
    } else {
        return 1;
    }
}

__global__ void gpu_bit_bucketing(char *buf, int *BUCKET, char *bitbucket, int *movie_key_list, int *index_movie_key_list, int *cur_idx_for_this_bucket, bool *bucket_seen) {
    
    int minIndex = threadIdx.x + blockIdx.x * 4;
    int maxIndex = minIndex + 4;
    int maxAllowed = *index_movie_key_list;
    
    for(int i = minIndex; i < maxIndex && i < maxAllowed; ++i) {
        if(bucket_seen[i]) continue;
        for(int j = minIndex; j < maxIndex && j < maxAllowed; ++j) {
            if(bucket_seen[j]) continue;

            int sa = movie_key_list[i] * 126;
            int ea = movie_key_list[i] * 126 + 126;
            
            int sb = movie_key_list[j] * 126;
            int eb = movie_key_list[j] * 126 + 126;
            
            bool same = true;
            
            for(int p = sa, q = sb; p < ea && q < eb; ++p, ++q) {
                same &= (bitbucket[p] == bitbucket[q]);
            }
            if(same) {
                //buf[i] = movie_key_list[j] + '0';
                
                atomicAdd(&(BUCKET[i * maxAllowed + cur_idx_for_this_bucket[i]]), movie_key_list[j]);
                atomicAdd(&cur_idx_for_this_bucket[i], 1);
                bucket_seen[j] = true;
                
                buf[i] = BUCKET[i * maxAllowed + cur_idx_for_this_bucket[i] - 1] + '0';
                //buf[i] = cur_idx_for_this_bucket[i] + '0';
            }
        }
    }
    buf[3] = '#';
    buf[4] = BUCKET[0] + '0';
    buf[5] = BUCKET[1] + '0';
    buf[6] = BUCKET[4] + '0';
    buf[7] = BUCKET[8] + '0';
}

int main()
{
    /* final.csv is sorted by movieId and then by userId. For each group of movieId the first 20 rows are picked. */
//    FILE* file = fopen("final.csv", "r");
    FILE* file = fopen("/kaggle/input/testset/test.csv", "r");
    if (file == NULL) {
        printf("Failed to open the file.\n");
        return 0;
    }
    memset(ratings, -1, sizeof(ratings));

    char line[MAX_LINE_LENGTH];
    int id_no = -1;
    while (fgets(line, sizeof(line), file)) {
        id_no++;
        if(id_no == 0) continue; // the first row has the names of the columns. omit that.
        int userId, movieId;
        int rating;
        sscanf(line, "%d,%d,%d", &userId, &movieId, &rating);
        if(movieId >= MAX_MOVIE_ID || userId >= MAX_USER_ID) continue;
        if(user[userId] == false) {
            user[userId] = true;
            unique_user += 1;
        }

        bitbucket[movieId][userId / 8] |= (1 << (userId % 8));
        if(movie[movieId] == false) {
            movie[movieId] = true;
            unique_movie += 1;
            movie_key_list[index_movie_key_list] = movieId;
            index_movie_key_list += 1;
        }

        //printf("%d %d %d\n", movieId, userId, rating);
        ratings[movieId][userId] = rating;
    }
    fclose(file);


    bool bucket_seen[index_movie_key_list];
    memset(bucket_seen, 0, sizeof(bucket_seen));

    int BUCKET[index_movie_key_list][index_movie_key_list];
    memset(BUCKET, 0, sizeof(BUCKET));

    int cur_idx_for_this_bucket[index_movie_key_list];
    memset(cur_idx_for_this_bucket, 0, sizeof(cur_idx_for_this_bucket));
    
    
    // parallel bit-bucketing-start
    
    int *dBUCKET, *dmovie_key_list, *dindex_movie_key_list, *dcur_idx_for_this_bucket;
    char *dbitbucket, *dbuf;
    bool *dbucket_seen;
    
    cudaMalloc((void **) &dBUCKET, index_movie_key_list * index_movie_key_list * sizeof(int));
    cudaMalloc((void **) &dbitbucket, MAX_MOVIE_ID * 126 * sizeof(char));
    cudaMalloc((void **) &dmovie_key_list, MAX_MOVIE_ID * sizeof(int));
    cudaMalloc((void **) &dcur_idx_for_this_bucket, MAX_MOVIE_ID * sizeof(int));
    cudaMalloc((void **) &dbucket_seen, index_movie_key_list * sizeof(bool));
    cudaMalloc((void **) &dindex_movie_key_list, sizeof(int));
    cudaMalloc((void **) &dbuf, 10 * 32 * sizeof(char));
    
    
    
    cudaMemcpy(dBUCKET, BUCKET, index_movie_key_list * index_movie_key_list * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dbitbucket, bitbucket, MAX_MOVIE_ID * 126 * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(dmovie_key_list, movie_key_list, MAX_MOVIE_ID * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dcur_idx_for_this_bucket, cur_idx_for_this_bucket, MAX_MOVIE_ID * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dbucket_seen, bucket_seen, index_movie_key_list * sizeof(bool), cudaMemcpyHostToDevice);
    cudaMemcpy(dindex_movie_key_list, &index_movie_key_list, sizeof(int), cudaMemcpyHostToDevice);
    
    // kernel call
    gpu_bit_bucketing<<<2, 1>>>(dbuf, dBUCKET, dbitbucket, dmovie_key_list, dindex_movie_key_list, dcur_idx_for_this_bucket, dbucket_seen);
    
    //int **test = (int **) malloc(index_movie_key_list * index_movie_key_list * sizeof(int));
    //cudaMemcpy(test, dBUCKET, index_movie_key_list * index_movie_key_list * sizeof(int), cudaMemcpyDeviceToHost);
    
    cudaDeviceSynchronize();
    
    cudaMemcpy(cur_idx_for_this_bucket, dcur_idx_for_this_bucket, MAX_MOVIE_ID * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(bucket_seen, dbucket_seen, index_movie_key_list * sizeof(bool), cudaMemcpyDeviceToHost);
    cudaMemcpy(BUCKET, dBUCKET, index_movie_key_list * index_movie_key_list * sizeof(int), cudaMemcpyDeviceToHost);
    
    char buf[10 * 32];
    cudaMemcpy(buf, dbuf, 10 * 32 * sizeof(char), cudaMemcpyDeviceToHost);
    
    cudaFree(dcur_idx_for_this_bucket);
    cudaFree(dbucket_seen);
    cudaFree(dBUCKET);
    
    // parallel-bit-bucketing-end
    
    // serial-bit-bucketing-start
    
    /*
    for(int i = 0; i < index_movie_key_list; ++i) {
        if(bucket_seen[i]) continue;
        for(int j = 0; j < index_movie_key_list; ++j) {
            if(bucket_seen[j]) continue;

            if(!memcmp(bitbucket[movie_key_list[i]], bitbucket[movie_key_list[j]], sizeof(bitbucket[0]))) {
                BUCKET[i][cur_idx_for_this_bucket[i]] = movie_key_list[j];
                cur_idx_for_this_bucket[i] += 1;
                bucket_seen[j] = true;
            }
        }
    } */
    
    // serial-bit-bucketing-end
    
    printf("Debug Buffer: %s\n", buf);
    for(int i = 0; i < 4; ++i) {
        for(int j = 0; j < 4; ++j) {
            printf("%d ", BUCKET[i][j]);
        }
        printf("\n");
    }
    
    for(int i = 0; i < 4; ++i) {
        printf("cur: %d", cur_idx_for_this_bucket[i]);
    }
    printf("\n");
    
    printf("%d %d %d %d\n", BUCKET[0][0], BUCKET[0][1], BUCKET[1][0], BUCKET[2][0]);
    for(int i = 0; i < index_movie_key_list; ++i) {
        
        if(bucket_seen[i] == false) continue;
        printf("No of movies in this bucket: %d\n", cur_idx_for_this_bucket[i]);
        printf("This bucket has the movieIds: \n");
        for(int j = 0; j < cur_idx_for_this_bucket[i]; ++j) {
            printf("%d ", BUCKET[i][j]);
        }
        printf("\n");
        
    }
    
    return 0;
}

Overwriting parallel-bit-bucketing.cu


In [12]:
!nvcc -o main parallel-bit-bucketing.cu
!./main

Debug Buffer: 423#1423
1 4 0 0 
2 0 0 0 
3 0 0 0 
0 0 0 0 
cur: 2cur: 1cur: 1cur: 0
1 4 2 3
No of movies in this bucket: 2
This bucket has the movieIds: 
1 4 
No of movies in this bucket: 1
This bucket has the movieIds: 
2 
No of movies in this bucket: 1
This bucket has the movieIds: 
3 
No of movies in this bucket: 0
This bucket has the movieIds: 

